> At the moment of writing, this agent is at the top of the leaderboard. (But there's a lot of luck involved and much can happen in the next couple weeks.)

The notebook at 
https://www.kaggle.com/glazed/evolving-finite-state-machines 
shows how to evolve the finite state machine

In [ ]:
%%writefile SurvivorBias.py
"""
Survivor Bias
Entry to Kaggle's RPS contest of 2020-2021
"""


import numpy as np
from numba import njit
import pickle
import os

@njit()
def filter_get_score(friend_move, foe_move):
    if (friend_move == foe_move):
        return(1) #ties+= 1
    else:
        wining_move = (foe_move + 1) % 3
        if (friend_move == wining_move):
            return(2) #wins += 1
        else:
            return(0) #losses += 1
    print('BADNESS in filter_get_score', friend_move, foe_move)
    return(-1000) # error condition

@njit()
def calc_markovish_index(action_seq): 
    I = 0
    for i in range(len(action_seq)):
        I = (I * 3) + action_seq[i]
    I = (I * 3)
    return I



 
@njit()
def predict(scores, det):
    # if scores are uniform, pick randomly, but if zero predict -1 -> abstain. 12/22/20
    if (np.sum(scores) < 0.001): return(-1)
    if (det): #predicted = np.argmax(scores)
        order = np.zeros(3, np.int32)
        order[0], order[1], order[2] = 0, 1, 2
        for i in range(3): # shuffle
            t = int(np.random.randint(3))
            order[i], order[t] = order[t], order[i]
        predicted = order[0]
        for i in range(3): # pick the max 
            if (scores[order[i]] > scores[predicted]): predicted = order[i]
        # maxs = np.where(scores == scores.max) # pick biggest but break scores randomly
        # predicted = int (np.choose(maxs) )
        # otherwise return a stochastic choice (Numba doesn't like np.random.choice())
    elif (np.sum(scores) > 0.01): 
        #np.random.choice(3, 1, p=list(scores/np.sum(scores)))
        scores = scores / np.sum(scores)
        if (np.abs( 1 - np.sum(scores) ) > 0.0001): print('BADNESS in filters.predict', np.sum(scores))
        v = np.random.random()
        predicted = 0
        if (v > scores[0]): predicted = 1
        if (v > scores[0] + scores[1]): predicted = 2
    else: predicted = int(np.random.randint(3))
    return(predicted)


    
@njit()
def get_hist_predictions_s(f, s, BFT, moves, k, howFarBack):
    """ return the prediction for this single-player history filter """
    # s -> step
    # BFT -> table
    # moves => history of moves for 1 player
    if (k == 0):
        scores = np.zeros(3, np.float32)
        I = 0
        for j in range(3): scores[j] = BFT [f, I + j] 
        return(scores)
        
    seq = np.zeros(k, np.int32)
    scores = np.zeros(3, np.float32)
    if (s > (2 * k) + 1 + howFarBack):
        for j in range(k):
            seq[j] = moves[(s - k) + j - howFarBack]
        I = calc_markovish_index(seq)
        for j in range(3): scores[j] = BFT [f, I + j] 
    return(scores)


@njit()
def get_hist_predictions_d(f, s, BFT, friend_moves, foe_moves, k, howFarBack):
    """ return the prediction for this dual-player history filter """
    seq = np.zeros(2*k, np.int32)
    scores = np.zeros(3, np.float32)
    if (s > (1 * k) + 1 + howFarBack):
        for j in range(k):
            seq[j    ] = friend_moves[(s - k) + j - howFarBack]
            seq[j + k] = foe_moves   [(s - k) + j - howFarBack]
        I = calc_markovish_index(seq)
        for j in range(3): scores[j] = BFT [f, I + j]
    return(scores)

 
@njit()
def update_bft_d(f, s, BFT, friend_moves, foe_moves, k, howFarBack, do_alpha = 0):
    if (do_alpha == 0): return(update_bft_d_alpha(f, s, BFT, friend_moves, foe_moves, k, howFarBack))
    seq = np.zeros(2*k, np.int32)
    move_to_predict = foe_moves[s]
    if (s > (1 * k) + 1 + howFarBack):
        for j in range(k):
            seq[j    ] = friend_moves[(s - k) + j - howFarBack]
            seq[j + k] = foe_moves   [(s - k) + j - howFarBack]
        I = calc_markovish_index(seq)
        # update table (BFT)
        BFT [f, I + move_to_predict] += 1
    return(BFT)
 
@njit()
def update_bft_d_alpha(f, s, BFT, friend_moves, foe_moves, k, howFarBack):
    seq = np.zeros(2*k, np.int32)
    move_to_predict = foe_moves[s]
    if (s > (1 * k) + 1 + howFarBack):
        for j in range(k):
            seq[j    ] = friend_moves[(s - k) + j - howFarBack]
            seq[j + k] = foe_moves   [(s - k) + j - howFarBack]
        I = calc_markovish_index(seq)
        # update table (BFT)
        alpha = 1
        beta  = 1 - alpha
        for n in range(3): BFT [f, I + n] *= beta
        BFT [f, I + move_to_predict] += 1 * alpha
    return(BFT)

@njit()
def update_bft_s_alpha(f, s, BFT, moves, k, howFarBack):
    move_to_predict = moves[s]
    if (k == 0):
        if (s > 0):
            I = 0
            # update table (BFT)
            BFT [f, I + move_to_predict] += 1
        return(BFT)
    seq = np.zeros(k, np.int32)
    if (s > (1 * k) + 1 + howFarBack):
        for j in range(k): seq[j] = moves[(s - k) + j - howFarBack]
        I = calc_markovish_index(seq)
        # update table (BFT)
        alpha = 1
        beta  = 1 - alpha
        for n in range(3): BFT [f, I + n] *= beta
        BFT [f, I + move_to_predict] += 1 * alpha
    return(BFT)

@njit()
def update_bft_s(f, s, BFT, moves, k, howFarBack, do_alpha = 0):
    if (do_alpha == 0): return(update_bft_s_alpha(f, s, BFT, moves, k, howFarBack))
    move_to_predict = moves[s]
    if (k == 0):
        if (s > 0):
            I = 0
            # update table (BFT)
            BFT [f, I + move_to_predict] += 1
        return(BFT)
    seq = np.zeros(k, np.int32)
    if (s > (1 * k) + 1 + howFarBack):
        for j in range(k): seq[j] = moves[(s - k) + j - howFarBack]
        I = calc_markovish_index(seq)
        # update table (BFT)
        BFT [f, I + move_to_predict] += 1
    return(BFT)

@njit()
def update_payoffs(predictions, payoffs, prev_foe_move):
    for f in range(len(predictions)):
        p = predictions[f]
        if (p != -1): # if it really weighed in
            payoffs[f, 3] += 1
            for i in range(3): # same as predicted, win over, lose to
                a = (p + i) % 3
                payoffs[f, i] += filter_get_score(a, prev_foe_move)
    return(payoffs)

@njit()
def relevance_of_payoffs(filters, payoffs):
    relevance = np.zeros(filters, np.int32)
    for f in range(filters):
        relevance[f] = np.max(payoffs[f,:]) - np.min(payoffs[f,:])
    return(relevance)



@njit()
def best_bet_voting(predictions, payoffs):
    rps = np.zeros(3, np.float32)
    action = (predictions[0] + 1) % 3
    for f in range(len(predictions)):
        p = predictions[f]
        if ((p != -1) and (payoffs[f, 3] > 0)):
            for i in range(3): # same as predicted, win over, lose to
                action = (p + i) % 3
                pay = payoffs[f, i] #/ payoffs[f, 3]
                rps [action] += pay #a vote for this action weighted by pay
    action = predict(rps, True) # pick best action, breaking ties randomly
    if (action == -1): action = np.random.randint(3)
    return(np.int32(action))


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        
@njit()
def create_great_filter():
    hist = 4 # maximum history
    # 8 basic filters
    #x = np.array([1, 2, 3, 0, 1, 2, 3, 4]) # histories to look back
    #m = np.array([0, 0, 0, 1, 1, 1, 1, 1]) # dual players -> 0, single player -> 1
    #add more, longer filters
    x = np.array([1, 2, 3, 0, 1, 2, 3, 4]) # histories to look back
    m = np.array([0, 0, 0, 1, 1, 1, 1, 1]) # dual players -> 0, single player -> 1
    # double that because I want a set predicting my move and another predicting his
    # and double that because I want a set decaying and another not. So 32 filtera
    # L = np.array([0, 1, 0, 1, 0, 1, 0, 1]) # predict me   -> 0, predict foe   -> 1
    # M = np.array([0, 0, 0, 0, 1, 1, 1, 1]) # dual players -> 0, single player -> 1
    dt = 1
    da = 2
    i2 = 2
    lx = len(x)
    filters = dt * da * i2 * lx
    K = np.zeros(filters, np.int32) # how many taps (history)
    L = np.zeros(filters, np.int32) # predict me   -> 0, predict foe   -> 1
    M = np.zeros(filters, np.int32) # dual players -> 0, single player -> 1
    A = np.zeros(filters, np.int32) # last-seen    -> 0, average       -> 1
    D = np.zeros(filters, np.int32) # howFarBack
    for d in range(dt):
        for i in range(i2):
            for j in range(lx):
                for a in range(da):
                    f = (d * i2 * lx * da) + (i * lx * da) + (j * da) + a
                    K[f] = x[j]
                    M[f] = m[j]
                    L[f] = i
                    D[f] = d
                    A[f] = a
    return(filters, hist, K, L, M, D, A)


#@njit()
def run_great_filters(s, friend_moves, foe_moves, FB, prev_predictions, payoffs, LF = None):
    filters, hist, K, L, M, D, A = create_great_filter()
    predicted = np.zeros(filters, np.int32) # predicted actions from the filters
    if (s == 0):
        table_size = int(pow(3, (2*hist) + 2))
        FB = np.zeros((filters, table_size), np.float32)
        payoffs = np.zeros((filters, 4), np.int32)
        
        action = np.random.randint(3)
        return (action, FB, predicted, payoffs)
    if (s > 0):
        # update tables (BFT)
        for f in range(filters):
            if (L[f] == 0):  # predicting foe
                if (M[f] == 0): FB = update_bft_d(f, s-1, FB, friend_moves, foe_moves, K[f], D[f], A[f]) # dual
                else:           FB = update_bft_s(f, s-1, FB,               foe_moves, K[f], D[f], A[f]) # single
            else:           # predicting me
                if (M[f] == 0): FB = update_bft_d(f, s-1, FB, foe_moves, friend_moves, K[f], D[f], A[f]) # dual
                else:           FB = update_bft_s(f, s-1, FB,            friend_moves, K[f], D[f], A[f]) # single
            
    # calculate the payoffs for the previous predictions   
    if (s >= 1): #(2 * min(k)) + 1): 
        prev_foe_move = foe_moves[s - 1]
        #payoffs = update_payoffs_alpha(prev_predictions, payoffs, prev_foe_move)
        payoffs = update_payoffs(prev_predictions, payoffs, prev_foe_move)
    
    # get predictions from all the filters
        for f in range(filters):
            if (L[f] == 0):   # predicting foe
                if (M[f] == 0): scores = get_hist_predictions_d(f, s, FB, friend_moves, foe_moves, K[f], D[f])# dual
                else:           scores = get_hist_predictions_s(f, s, FB,               foe_moves, K[f], D[f])# single
            else:             # predicting me
                if (M[f] == 0): scores = get_hist_predictions_d(f, s, FB, foe_moves, friend_moves, K[f], D[f]) # dual
                else:           scores = get_hist_predictions_s(f, s, FB,            friend_moves, K[f], D[f]) # single
            if (LF != None):
                LF.write(' (')
                for e in scores: LF.write(str(e) + ',')
                LF.write(')')
            predicted[f] = predict(scores, True)
    
    if (LF != None):
        LF.write(' pay ')
        for pp in pretty_payoffs(filters, payoffs):
            LF.write(' (')
            for e in pp:LF.write(str(e) + ',')
            LF.write(')')
                
    if (s < 8): action = np.random.randint(3)
    else:    
        action = best_bet_voting(predicted, payoffs) 
    return(action, FB, predicted, payoffs)

@njit()
def pretty_payoffs(num_filters, payoffs):
    """ return a list of lists of payoffs for each filter """
    pretty = []
    for f in range(num_filters):
        pay = []
        for i in range(3): # same as predicted, win over, lose to
            pay.append( payoffs[f, i] )
        pretty.append(pay)
    return(np.array(pretty))
    

@njit()
def sloppy_init():
    FB = np.zeros((2, 2), np.float32)
    predicted = np.zeros(2, np.int32) # predicted actions from the filters
    payoffs   = np.ones((2, 4), np.int32)
    return(FB, predicted, payoffs) 


def FSM_agentX(step, prev_friend_act, prev_foe_act, state, filter_move, state_choices, action_choices):
    """ play 1 move using finite state machine """
    # num_actions == 3
    holdoff = -1 # spin at state zero until past this step
# =============================================================================
#     4 possible actions:
#       filter_move
#       counter to filter_move
#       counter-counter to filter_move
#       random
#     3 possible conditions: win, lose, tie
# =============================================================================
    showit = False
    if (showit): 
        print('FSM_agentX() s', state)
    if step == 0:
        condition = 2 # tie
        if (state != 0):
            print("FSM_agentX, starting state =", state)
    if step <= holdoff:
        condition = 2 # tie
        #    action = np.int32(np.random.randint(3))
        state = state_choices[state, condition] # get next state
        #a = np.int32(action_choices[state]) # get action to play
        action = np.int32(np.random.randint(3))
        state = 0 # remain at state 0
        if (showit): print('FSM_agentX() step 0. action', action, 'state', state)
        return (state, action)
    
    # find new state based on condion (win, lose, tie)
    if (prev_friend_act == prev_foe_act): condition = 2 # tie
    elif (prev_friend_act == (prev_foe_act +1) % 3): condition = 0 # win
    else: condition = 1 # lose
    
    #if (condition != 0): #only change state on a loss or tie
    state = state_choices[state, condition] # get next state
    a = np.int32(action_choices[state]) # get action to play
        
    if (a < 3): # pick action relative to filter_move
        action = (filter_move + a) % 3
    else: action = int(np.random.randint(3))
    if (showit): 
        print('FSM_agentX() state', state, 'action', action )
    return (state, action)       


class INDIVIDUAL():
    """
    This class holds an individual for a GA.
    It represents a finite state machine to play Roshambo.
    """
    def __init__(self):
        self.N = 200 # number of states
        self.action_choices = np.zeros((self.N), np.int32) # coded state table (0-4)
        self.state_choices = np.zeros((self.N, 3), np.int32) # coded state table (0-N)
  
F =  INDIVIDUAL() 
F.action_choices = pickle.loads(b'\x80\x04\x95\xad\x03\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\xc8\x85\x94h\x03\x8c\x05dtype\x94\x93\x94\x8c\x02i4\x94K\x00K\x01\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B \x03\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x94t\x94b.')
F.state_choices  = pickle.loads(b'\x80\x04\x95\xef\t\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\xc8K\x03\x86\x94h\x03\x8c\x05dtype\x94\x93\x94\x8c\x02i4\x94K\x00K\x01\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B`\t\x00\x00D\x00\x00\x00^\x00\x00\x00 \x00\x00\x00\x11\x00\x00\x00\x17\x00\x00\x00\xc4\x00\x00\x00;\x00\x00\x00e\x00\x00\x00<\x00\x00\x00y\x00\x00\x00 \x00\x00\x00$\x00\x00\x00M\x00\x00\x00\x80\x00\x00\x003\x00\x00\x00\x95\x00\x00\x001\x00\x00\x00\x8a\x00\x00\x00\x8e\x00\x00\x00H\x00\x00\x00J\x00\x00\x00n\x00\x00\x00\x00\x00\x00\x00Z\x00\x00\x00\t\x00\x00\x00\xb6\x00\x00\x00\x10\x00\x00\x00\xaf\x00\x00\x00z\x00\x00\x00\x07\x00\x00\x00\x01\x00\x00\x00)\x00\x00\x00\x0b\x00\x00\x00^\x00\x00\x00\xc5\x00\x00\x00\x90\x00\x00\x00\xa0\x00\x00\x00h\x00\x00\x00\xbb\x00\x00\x00\x0e\x00\x00\x00o\x00\x00\x00f\x00\x00\x00\x90\x00\x00\x00\xc2\x00\x00\x00\xc5\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00}\x00\x00\x00$\x00\x00\x00C\x00\x00\x00P\x00\x00\x00\x91\x00\x00\x00"\x00\x00\x00\xad\x00\x00\x00\xb8\x00\x00\x00\xb9\x00\x00\x005\x00\x00\x00f\x00\x00\x00\xac\x00\x00\x00\xad\x00\x00\x00\xae\x00\x00\x00\x87\x00\x00\x00q\x00\x00\x00\x9f\x00\x00\x00\xbe\x00\x00\x00b\x00\x00\x00\x04\x00\x00\x00R\x00\x00\x00C\x00\x00\x00t\x00\x00\x00\x80\x00\x00\x00\x84\x00\x00\x00A\x00\x00\x00%\x00\x00\x00\x04\x00\x00\x00\xa9\x00\x00\x00d\x00\x00\x004\x00\x00\x00`\x00\x00\x00V\x00\x00\x00t\x00\x00\x00\xb8\x00\x00\x00,\x00\x00\x00\x8d\x00\x00\x00>\x00\x00\x00\xa3\x00\x00\x00C\x00\x00\x00a\x00\x00\x00\x8c\x00\x00\x00e\x00\x00\x00h\x00\x00\x00B\x00\x00\x00\xb3\x00\x00\x00\x9d\x00\x00\x00Z\x00\x00\x00\xa2\x00\x00\x00,\x00\x00\x00g\x00\x00\x00/\x00\x00\x00D\x00\x00\x00 \x00\x00\x00B\x00\x00\x00%\x00\x00\x00\x00\x00\x00\x003\x00\x00\x00j\x00\x00\x00l\x00\x00\x00\xa8\x00\x00\x005\x00\x00\x00\xb4\x00\x00\x00S\x00\x00\x00?\x00\x00\x00 \x00\x00\x00>\x00\x00\x00\x91\x00\x00\x00^\x00\x00\x00\xc4\x00\x00\x00\xb1\x00\x00\x00\x03\x00\x00\x00\xac\x00\x00\x00\x90\x00\x00\x00,\x00\x00\x00\x0f\x00\x00\x00\x84\x00\x00\x002\x00\x00\x00\x93\x00\x00\x00/\x00\x00\x00\x8a\x00\x00\x00q\x00\x00\x00\xa1\x00\x00\x00\x90\x00\x00\x00\'\x00\x00\x00z\x00\x00\x00\xa0\x00\x00\x00\xa3\x00\x00\x00\x16\x00\x00\x00T\x00\x00\x00\x9e\x00\x00\x00\x1f\x00\x00\x00A\x00\x00\x00\x8e\x00\x00\x00\x03\x00\x00\x00\x92\x00\x00\x00A\x00\x00\x00\\\x00\x00\x00B\x00\x00\x00@\x00\x00\x00|\x00\x00\x00\xb3\x00\x00\x00f\x00\x00\x00\x05\x00\x00\x003\x00\x00\x00\x91\x00\x00\x00\x8d\x00\x00\x00~\x00\x00\x00H\x00\x00\x00n\x00\x00\x00\x08\x00\x00\x00l\x00\x00\x00\xc1\x00\x00\x00>\x00\x00\x00/\x00\x00\x00|\x00\x00\x00\x92\x00\x00\x00\xc0\x00\x00\x00\x9f\x00\x00\x00\x11\x00\x00\x000\x00\x00\x00\x06\x00\x00\x007\x00\x00\x00y\x00\x00\x00\xa2\x00\x00\x00\x98\x00\x00\x00\xc5\x00\x00\x00\x18\x00\x00\x00\xb6\x00\x00\x000\x00\x00\x00\x97\x00\x00\x00\x01\x00\x00\x00c\x00\x00\x00\xa8\x00\x00\x00-\x00\x00\x00u\x00\x00\x00\xb6\x00\x00\x00\xba\x00\x00\x00\x06\x00\x00\x00W\x00\x00\x009\x00\x00\x00m\x00\x00\x00\x9d\x00\x00\x00\x89\x00\x00\x00\x1b\x00\x00\x00\xb7\x00\x00\x00(\x00\x00\x00+\x00\x00\x00W\x00\x00\x00i\x00\x00\x00\xc6\x00\x00\x00\xc3\x00\x00\x00\xa4\x00\x00\x00X\x00\x00\x007\x00\x00\x00w\x00\x00\x009\x00\x00\x00\x0c\x00\x00\x00\xaf\x00\x00\x00@\x00\x00\x00\x8c\x00\x00\x00z\x00\x00\x00@\x00\x00\x00v\x00\x00\x00.\x00\x00\x00A\x00\x00\x00\x08\x00\x00\x00\x1a\x00\x00\x00\xb7\x00\x00\x00&\x00\x00\x00\'\x00\x00\x00\'\x00\x00\x00\x17\x00\x00\x00f\x00\x00\x00\xb4\x00\x00\x00e\x00\x00\x00\x17\x00\x00\x00\xa5\x00\x00\x00?\x00\x00\x00\xaf\x00\x00\x00E\x00\x00\x00\xa2\x00\x00\x00E\x00\x00\x00N\x00\x00\x00\xa2\x00\x00\x00\xb6\x00\x00\x00D\x00\x00\x00\xaa\x00\x00\x00}\x00\x00\x00\'\x00\x00\x00\x9d\x00\x00\x00\x85\x00\x00\x00\xa6\x00\x00\x00\'\x00\x00\x00k\x00\x00\x00\xbf\x00\x00\x00G\x00\x00\x00\x0c\x00\x00\x00\x10\x00\x00\x00\x89\x00\x00\x00=\x00\x00\x00z\x00\x00\x00U\x00\x00\x00\x80\x00\x00\x00\x93\x00\x00\x00=\x00\x00\x00\xbd\x00\x00\x00\t\x00\x00\x00,\x00\x00\x00r\x00\x00\x00\x0b\x00\x00\x00X\x00\x00\x00\x8d\x00\x00\x00K\x00\x00\x00y\x00\x00\x00\xb7\x00\x00\x00\xaf\x00\x00\x00M\x00\x00\x00s\x00\x00\x00\t\x00\x00\x009\x00\x00\x00\x03\x00\x00\x00T\x00\x00\x00i\x00\x00\x00@\x00\x00\x00k\x00\x00\x00\x93\x00\x00\x00\x9d\x00\x00\x00~\x00\x00\x00^\x00\x00\x00\xa8\x00\x00\x00r\x00\x00\x00\x94\x00\x00\x00\x9e\x00\x00\x00&\x00\x00\x00m\x00\x00\x001\x00\x00\x00}\x00\x00\x00?\x00\x00\x00r\x00\x00\x00?\x00\x00\x00\x13\x00\x00\x00%\x00\x00\x00\x1b\x00\x00\x00R\x00\x00\x00:\x00\x00\x00}\x00\x00\x00\x01\x00\x00\x00\xc5\x00\x00\x00e\x00\x00\x00\x17\x00\x00\x00\x9e\x00\x00\x00\x1f\x00\x00\x00Z\x00\x00\x00G\x00\x00\x00\xc0\x00\x00\x00^\x00\x00\x00}\x00\x00\x00\xa6\x00\x00\x00Z\x00\x00\x00P\x00\x00\x00\xb3\x00\x00\x00\\\x00\x00\x00\xc5\x00\x00\x00\x9e\x00\x00\x00\xb8\x00\x00\x00M\x00\x00\x007\x00\x00\x00\x8d\x00\x00\x00\xa3\x00\x00\x00D\x00\x00\x00\x8d\x00\x00\x00\x9a\x00\x00\x00=\x00\x00\x00\x01\x00\x00\x00\x14\x00\x00\x00 \x00\x00\x00\xb7\x00\x00\x00\x0c\x00\x00\x007\x00\x00\x00B\x00\x00\x00<\x00\x00\x00\xc6\x00\x00\x00j\x00\x00\x00U\x00\x00\x00~\x00\x00\x00@\x00\x00\x00\x81\x00\x00\x00\xb2\x00\x00\x00&\x00\x00\x00G\x00\x00\x00s\x00\x00\x00R\x00\x00\x00\x96\x00\x00\x00\xb2\x00\x00\x00&\x00\x00\x00\x9d\x00\x00\x00\x0c\x00\x00\x00"\x00\x00\x003\x00\x00\x00\x00\x00\x00\x00\x8d\x00\x00\x00\xc5\x00\x00\x00\x01\x00\x00\x00\xa3\x00\x00\x00\x99\x00\x00\x00c\x00\x00\x00$\x00\x00\x00\x05\x00\x00\x00\xad\x00\x00\x00\xb0\x00\x00\x00\x13\x00\x00\x00U\x00\x00\x00\xab\x00\x00\x00\xb7\x00\x00\x00]\x00\x00\x00Y\x00\x00\x00"\x00\x00\x00\x89\x00\x00\x00~\x00\x00\x00\x93\x00\x00\x00q\x00\x00\x00\xc1\x00\x00\x00b\x00\x00\x00\x18\x00\x00\x00,\x00\x00\x00~\x00\x00\x00\x90\x00\x00\x00\xa2\x00\x00\x00V\x00\x00\x00r\x00\x00\x00\x07\x00\x00\x00\'\x00\x00\x00M\x00\x00\x00\x11\x00\x00\x00Y\x00\x00\x00`\x00\x00\x00\x9f\x00\x00\x00\xa0\x00\x00\x00z\x00\x00\x00\xbc\x00\x00\x00\x8c\x00\x00\x00q\x00\x00\x00/\x00\x00\x00\x89\x00\x00\x00\xb5\x00\x00\x002\x00\x00\x00\x1c\x00\x00\x00n\x00\x00\x00\x93\x00\x00\x00\t\x00\x00\x00\xa9\x00\x00\x00\x83\x00\x00\x00\x81\x00\x00\x00\x16\x00\x00\x00\xb3\x00\x00\x00\xc0\x00\x00\x00%\x00\x00\x00]\x00\x00\x00X\x00\x00\x00\x07\x00\x00\x00/\x00\x00\x00\xc3\x00\x00\x00\x99\x00\x00\x00\x06\x00\x00\x00A\x00\x00\x00P\x00\x00\x00Q\x00\x00\x00@\x00\x00\x00\x92\x00\x00\x00\xc7\x00\x00\x00\x94\x00\x00\x00\r\x00\x00\x00\xc2\x00\x00\x00\x11\x00\x00\x002\x00\x00\x00R\x00\x00\x00i\x00\x00\x00\xbd\x00\x00\x00e\x00\x00\x00\xbd\x00\x00\x00\x91\x00\x00\x00O\x00\x00\x00\x93\x00\x00\x00\x19\x00\x00\x00o\x00\x00\x00\x97\x00\x00\x00"\x00\x00\x00\x0e\x00\x00\x00A\x00\x00\x00\n\x00\x00\x00:\x00\x00\x00 \x00\x00\x00\t\x00\x00\x00\x14\x00\x00\x00\xae\x00\x00\x00>\x00\x00\x00\x92\x00\x00\x00\x16\x00\x00\x00E\x00\x00\x00\xa7\x00\x00\x00\x1d\x00\x00\x00h\x00\x00\x00B\x00\x00\x00\x10\x00\x00\x00w\x00\x00\x00m\x00\x00\x00,\x00\x00\x00P\x00\x00\x00\x90\x00\x00\x00\xb2\x00\x00\x00\xab\x00\x00\x00o\x00\x00\x00\xc4\x00\x00\x00\x13\x00\x00\x004\x00\x00\x00D\x00\x00\x00o\x00\x00\x00\x1d\x00\x00\x00+\x00\x00\x00\x03\x00\x00\x00`\x00\x00\x008\x00\x00\x00\n\x00\x00\x00n\x00\x00\x00<\x00\x00\x001\x00\x00\x002\x00\x00\x00\x1a\x00\x00\x00"\x00\x00\x00{\x00\x00\x00\x18\x00\x00\x00!\x00\x00\x00\xb4\x00\x00\x003\x00\x00\x00\x8d\x00\x00\x00c\x00\x00\x00\'\x00\x00\x00\x07\x00\x00\x00\x9e\x00\x00\x00\x1f\x00\x00\x00\xc5\x00\x00\x00\xc2\x00\x00\x00\xb1\x00\x00\x00L\x00\x00\x00m\x00\x00\x00=\x00\x00\x00~\x00\x00\x00\x83\x00\x00\x00\x9c\x00\x00\x00L\x00\x00\x003\x00\x00\x00\xb5\x00\x00\x00\x9c\x00\x00\x00.\x00\x00\x00[\x00\x00\x00\n\x00\x00\x00\x19\x00\x00\x00~\x00\x00\x00\xc1\x00\x00\x00\x9f\x00\x00\x00u\x00\x00\x00\x98\x00\x00\x00\x0b\x00\x00\x00\xc1\x00\x00\x00\xba\x00\x00\x00\x82\x00\x00\x00\x83\x00\x00\x00\x02\x00\x00\x00\x85\x00\x00\x00\x18\x00\x00\x00\t\x00\x00\x00\x80\x00\x00\x00~\x00\x00\x00\x94\x00\x00\x00\xbf\x00\x00\x00F\x00\x00\x00\x0c\x00\x00\x00*\x00\x00\x00\x00\x00\x00\x00u\x00\x00\x00.\x00\x00\x00\xbb\x00\x00\x003\x00\x00\x00\xaa\x00\x00\x00o\x00\x00\x00\xaf\x00\x00\x00\xb5\x00\x00\x00q\x00\x00\x00\x02\x00\x00\x00D\x00\x00\x00\x8f\x00\x00\x00\xab\x00\x00\x000\x00\x00\x00\xa4\x00\x00\x00\xc5\x00\x00\x00y\x00\x00\x00\x08\x00\x00\x00\xbf\x00\x00\x00s\x00\x00\x00w\x00\x00\x00\x99\x00\x00\x00\xbf\x00\x00\x00\x07\x00\x00\x00\x1e\x00\x00\x00\x0c\x00\x00\x00\x90\x00\x00\x00[\x00\x00\x00\x94\x00\x00\x00\xa3\x00\x00\x00"\x00\x00\x00\x0c\x00\x00\x00L\x00\x00\x00\x11\x00\x00\x00w\x00\x00\x00\xa3\x00\x00\x00\xac\x00\x00\x00\xbb\x00\x00\x00\xc7\x00\x00\x00a\x00\x00\x00\xaa\x00\x00\x00\xbf\x00\x00\x00\xc1\x00\x00\x00\xa9\x00\x00\x00\xab\x00\x00\x00_\x00\x00\x007\x00\x00\x002\x00\x00\x00\x90\x00\x00\x00R\x00\x00\x00>\x00\x00\x00R\x00\x00\x00\x8c\x00\x00\x00\x03\x00\x00\x005\x00\x00\x00\xc6\x00\x00\x00\xc2\x00\x00\x00\x8d\x00\x00\x00$\x00\x00\x00\x95\x00\x00\x00\xa6\x00\x00\x003\x00\x00\x00\xa0\x00\x00\x00\xaa\x00\x00\x00]\x00\x00\x00\x17\x00\x00\x00Y\x00\x00\x00\t\x00\x00\x00j\x00\x00\x00!\x00\x00\x005\x00\x00\x00#\x00\x00\x00\x14\x00\x00\x00\x94t\x94b.')


 
def survivor_bias(observation, configuration):
    """ Main """
    global Gprev_friend_move, Gprev_foe_move, GFB, Gfriend_moves, Gfoe_moves, Gpredicted, Gpayoffs, Gstate
    trials = 1000
    s = observation.step
    
    if (s == 0):
        Gstate = 0
        # make some dummy sizes
        GFB, Gpredicted, Gpayoffs = sloppy_init()
        Gprev_foe_move = -1
        Gprev_friend_move = -1
        Gfriend_moves = np.zeros(trials, np.int32)
        Gfoe_moves    = np.zeros(trials, np.int32)
    if (s > 0):
        Gprev_foe_move = observation.lastOpponentAction
        Gfriend_moves[s - 1] = Gprev_friend_move
        Gfoe_moves   [s - 1] = Gprev_foe_move
    # Call  run_great_filters() to get a prediction for the best move to play
    # This routine takes 'forcing moves' and can easily be added to an ensemble
    filter_move, GFB, Gpredicted, Gpayoffs = run_great_filters(s, Gfriend_moves, Gfoe_moves, GFB, Gpredicted, Gpayoffs)
    # But the filter_move is passed to the evolved finite state machine, which treats it as merely a suggestion.
    # The finite state machine won't work in an ensemble
    Gstate, Gfriend_move = FSM_agentX(s, Gprev_friend_move, Gprev_foe_move, Gstate, filter_move, F.state_choices, F.action_choices) 
    Gprev_friend_move = Gfriend_move
        
    return int(Gfriend_move)